# Numerical Investigations to Burger's Equation

In [ ]:
import numpy as np
import matplotlib as plt

In [ ]:
import crank_nicholson as cn

Burger's equation is a non-linear PDE that allows one to develop intuition regarding the physical mechanisms of both advection and diffusion. The equation in one dimension is given by

\\[\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}.\\]

This can be thought of as a simplified Navier-Stokes equation with no body force terms or pressure gradient. $\nu$ represents the kinematic viscosity and thus the term $\nu (\partial^2 u/\partial x^2)$ represents the diffusion of viscous forces. The term $u (\partial u / \partial x)$ is the advection term, as seen in the Navier-Stokes equation. Often it is of interest to solve the problem when $\nu = 0$ - this is referred to as the inviscid Burger's equation, and is often expressed as

\\[\frac{\partial u}{\partial t} + \frac{1}{2}\frac{\partial}{\partial x}u^2=0.\\]

To develop intuition regarding the role of each type of transport, the viscid Burger's equation is first considered. The explicit finite difference method will be derived and implemented, though as with most explicit methods, the stability varies due to the accumulation of error with each step. 

### Explicit Finite Difference Method

The explicit method is obtained by using a forward-difference on the time derivative, meaning

\\[\frac{\partial u}{\partial t} \approx \frac{u(x,t+\Delta t) - u(x,t)}{\Delta t} = \frac{u_{i,j+1} - u_{i,j}}{\Delta t}.\\]

As seen in Landau, Paez and Bordeianu, the central-difference approximation is used on the diffusion term, resulting in

\\[\frac{\partial^2 u}{\partial x^2} \approx \frac{u(x+\Delta x,t) - 2u(x,t) + u(x-\Delta x,t)}{(\Delta x)^2} 
= \frac{u_{i+1,j} - 2u_{i,j} + u_{i-1,j}}{(\Delta x)^2}.\\]


There are a couple of different first order approaches to approximate the advection term. The first method uses a central-difference to approximate $\partial u / \partial x$. 

\\[\frac{\partial u}{\partial x} \approx \frac{u(x+\Delta x/2,t) - u(x-\Delta x / 2,t)}{\Delta x} = \frac{(u_{i+1,j}+u_{i,j})/2 - (u_{i-1,j}+u_{i,j})/2}{\Delta x} = \frac{u_{i+1,j} -u_{i-1,j}}{2\Delta x}\\]

Using this approximation, an explicit finite difference method for Burger's equation is given by 

\\[\begin{align} \frac{u_{i,j+1} - u_{i,j}}{\Delta t} + \frac{u_{i,j}(u_{i+1,j} -u_{i-1,j})}{2\Delta x} &= \frac{u_{i+1,j} - 2u_{i,j} + u_{i-1,j}}{(\Delta x)^2} \\ 
\Rightarrow \frac{u_{i,j+1} - u_{i,j}}{\Delta t} &= -\frac{u_{i,j}(u_{i+1,j} -u_{i-1,j})}{2\Delta x} + \frac{u_{i+1,j} - 2u_{i,j} + u_{i-1,j}}{(\Delta x)^2} \\
\Rightarrow u_{i,j+1} &= u_{i,j} + \Delta t\left( -\frac{u_{i,j}(u_{i+1,j} -u_{i-1,j})}{2\Delta x} + \frac{u_{i+1,j} - 2u_{i,j} + u_{i-1,j}}{(\Delta x)^2}\right)\end{align} \\]

In [ ]:
### Here we will import the function from finite.py #####

### Analytic Solutions for the Viscid Burger's Equation
From Inan and Bahadir, a more complex numerical method was used to solve Burger's equation, though the analytic solutions provide a way to compare the with the numerical solutions obtained above. From Table 3, 

| x    | t    | $\nu=1.0$ | $\nu=.01$ |
|------|------|-----------|------------|
| 0.25 | 0.10 | 0.253638  | 0.566328   |
|      | 0.15 | 0.156601  | 0.512148   |
|      | 0.20 | 0.096442  | 0.466583   |
|      | 0.25 | 0.059218  | 0.427995   |
|      |      |           |            |
| 0.50 | 0.10 | 0.371577  | 0.947414   |
|      | 0.15 | 0.226824  | 0.900098   |
|      | 0.20 | 0.138473  | 0.848365   |
|      | 0.25 | 0.084538  | 0.796762   |
|      |      |           |            |
| 0.75 | 0.10 | 0.272582  | 0.860134   |
|      | 0.15 | 0.164369  | 0.922756   |
|      | 0.20 | 0.099435  | 0.961891   |
|      | 0.25 | 0.060347  | 0.974689   |

which is for the problem with the following initial condition and boundary conditions

\\[u(x,0) = \mbox{sin}(\pi x) \qquad u(0,t)=u(1,t)=0\\]

with the analytical solution being given by

\\[u(x,t) = 2 \pi \nu \frac{\sum_{n=1}^\infty a_n \mbox{exp}(-n^2 \pi^2 \nu t)n \mbox{sin}(n\pi x)}{a_0 + \sum_{n=1}^\infty a_n \mbox{exp}(-n^2 \pi^2 \nu t)\mbox{cos}(n \pi x)}\\]

where 

\\[\begin{align*} 
a_0 &= \int_0^1 \mbox{exp}\left(-(1-\mbox{cos}(\pi x) )/(2 \pi \nu)\right)dx  \\
a_n &= 2\int_0^1 \mbox{exp}\left(-(1-\mbox{cos}(\pi x) )/(2 \pi \nu)\right)\mbox{cos}(n \pi x)dx.
\end{align*}\\]

## Crank-Nicholson Time-Step

As we saw, the explicit finite different method is only stable when $\Delta t$ is small compared to $(\Delta x)^2$. To implement this, the Cole-Hopf transformation is applied. This says that there is a function $\phi(x,t)$ such that

$$u(x,t) = -2 \nu \frac{1}{\phi}\frac{\partial \phi}{\partial x}.$$

This then allows us to write 

$$\begin{align*}
\frac{\partial u}{\partial t} &= -2\nu \frac{\partial }{\partial x}\left(\frac{1}{\phi}\frac{\partial \phi}{\partial t}\right) \\
\frac{1}{2}\frac{\partial u^2}{\partial x} &= 2\nu^2 \frac{\partial}{\partial x}\left(\frac{1}{\phi}\frac{\partial \phi}{\partial x}\right)^2 \\
\frac{\partial^2 u}{\partial x^2} &= 2\nu\frac{\partial}{\partial x}\left(\left(\frac{1}{\phi}\frac{\partial \phi}{\partial x}\right)^2 - \frac{1}{\phi}\frac{\partial^2 \phi}{\partial x^2}\right)
\end{align*}$$

meaning 

$$\begin{align*}-2\nu \frac{\partial }{\partial x}\left(\frac{1}{\phi}\frac{\partial \phi}{\partial t}\right) + 2\nu^2 \frac{\partial}{\partial x}\left(\frac{1}{\phi}\frac{\partial \phi}{\partial x}\right)^2&=  2\nu^2\frac{\partial}{\partial x}\left(\left(\frac{1}{\phi}\frac{\partial \phi}{\partial x}\right)^2 - \frac{1}{\phi}\frac{\partial^2 \phi}{\partial x^2}\right) \\
\Rightarrow  \frac{\partial }{\partial x}\left(\frac{1}{\phi}\frac{\partial \phi}{\partial t}\right) & = \nu\frac{\partial}{\partial x}\left( \frac{1}{\phi}\frac{\partial^2 \phi}{\partial x^2}\right).\end{align*}$$

By integrating both sides, we then obtain

$$\frac{1}{\phi}\frac{\partial \phi}{\partial t}  = \nu \frac{1}{\phi}\frac{\partial^2 \phi}{\partial x^2} + g(t).$$

But for now we are only interested in problems with time-independent boundary conditions, so we set $g(t)=0$. After applying this transformation, it is now apparent that the problem reduces to the diffusion equation

$$\frac{\partial \phi}{\partial t}  = \nu \frac{\partial^2 \phi}{\partial x^2}.$$

If we begin by considering problems in which $u(0,t) = u(L,t) = 0$, we see that the boundary conditions on $phi$ require $\phi_x(0,t) = \phi_x(L,t) = 0$. Following Kadalbajoo and Awasthi, applying the Crank-Nicholson time-stepping method with Neumann boundary conditions leads to the following system of equations:

$$\begin{align*}
(1+\alpha)\phi_{0,j+1} -\alpha \phi_{1,j+1} &= \alpha \phi_{1,j} + (1-\alpha)\phi_{0,j} \\
-\frac{\alpha}{2}\phi_{i-1,j+1} + (1+\alpha)\phi_{i,j+1}-\frac{\alpha}{2}\phi_{i+1,j+1} &=
\frac{\alpha}{2}\phi_{i-1,j} + (1-\alpha)\phi_{i,j}+\frac{\alpha}{2}\phi_{i+1,j} \qquad \mbox{when }1 \leq i \leq M\\
-\alpha\phi_{M-1,j+1} + (1+\alpha)\phi_{M,j+1} &= \alpha \phi_{M-1,j}+(1-\alpha) \phi_{M,j}
\end{align*}$$

This system of equations can then be written using tridiagonal matrices.
$$\begin{pmatrix}
            1+\alpha          & -\alpha           & 0                & ...\\
            -\frac{\alpha}{2} & 1+\alpha          &-\frac{\alpha}{2} & ... \\
            \vdots            & \vdots            &   \vdots         & ... \\
            0                 & -\frac{\alpha}{2} & 1+\alpha         &-\frac{\alpha}{2} \\
            0                 & 0                 &-\alpha           & 1+\alpha \\
   \end{pmatrix}
   \begin{pmatrix}
   \phi_{0,j+1} \\
   \phi_{1,j+1} \\
   \vdots    \\
   \phi_{M-1,j+1}\\
   \phi_{M,j+1}\\
   \end{pmatrix}
   =
   \begin{pmatrix}
            1-\alpha          & \alpha            & 0                & ...\\
            \frac{\alpha}{2}  & 1-\alpha          &\frac{\alpha}{2}  & ... \\
            \vdots            & \vdots            &   \vdots         & ... \\
            0                 &  \frac{\alpha}{2} & 1-\alpha         & \frac{\alpha}{2} \\
            0                 & 0                 & \alpha           & 1-\alpha \\
   \end{pmatrix}
   \begin{pmatrix}
   \phi_{0,j} \\
   \phi_{1,j} \\
   \vdots    \\
   \phi_{M-1,j}\\
   \phi_{M,j}\\
   \end{pmatrix}
   $$

To solve this system of equations, the method outlined in section 20.4.1 of Landau, et al. is used. A function was written to generate the above tridiagonal matrices, and a tridiagonal matrix solver was written. Before using this in the PDE solver, a couple checks were run to ensure that the solver works correctly. 

In [ ]:
import numpy.testing as test

alpha = 0.5

# cn.init_matrices()

### Maybe remove this later, or run simulation with this approximation instead and compare

The advection term can then be approximated using Taylor expansions of both $u^2(x+\Delta x,t)/2$ and $u^2(x-\Delta x,t)/2$ as shown

\\[\frac{u^2(x+\Delta x,t)}{2} = \frac{u^2}{2} + \frac{2u}{2}\frac{\partial u}{\partial x}\Delta x+...=\frac{u^2}{2} + u\frac{\partial u}{\partial x}\Delta x+...\\]

\\[\frac{u^2(x-\Delta x,t)}{2} = \frac{u^2}{2} - \frac{2u}{2}\frac{\partial u}{\partial x}\Delta x+...=\frac{u^2}{2} - u\frac{\partial u}{\partial x}\Delta x+..\\]

Thus

\\[\begin{align*}
\frac{u^2(x+\Delta x,t)}{2} - \frac{u^2(x-\Delta x,t)}{2} &\approx \left(\frac{u^2}{2} + u\frac{\partial u}{\partial x}\Delta x\right) - \left(\frac{u^2}{2} - u\frac{\partial u}{\partial x}\Delta x\right) \\
&= 2 u \frac{\partial u}{\partial x}\Delta x
\end{align*} \\]

meaning that
\\[u \frac{\partial u}{\partial x} \approx \frac{1}{2\Delta x}\left(\frac{u^2(x+\Delta x,t)}{2} - \frac{u^2(x-\Delta x,t)}{2}\right) = \frac{1}{4}\left(\frac{u_{i+1,j}^2 - u_{i-1,j}^2}{\Delta x}\right).\\]

### References

1. Inan, Bilge, and Ahmet Refik Bahadir. “Numerical Solution of the One-Dimensional Burgers’ Equation: Implicit and Fully Implicit Exponential Finite Difference Methods.” Pramana, vol. 81, no. 4, 2013, pp. 547–556., doi:10.1007/s12043-013-0599-z. 

2. Kadalbajoo, Mohan. K., and A. Awasthi. “A Numerical Method Based on Crank-Nicolson Scheme for Burgers’ Equation.” Applied Mathematics and Computation, vol. 182, no. 2, 2006, pp. 1430–1442., doi:10.1016/j.amc.2006.05.030. 

3. Landau, Rubin H., et al. Computational Physics: Problem Solving with Python. Wiley-VCH, 2015. 